In [104]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import re

import matplotlib.pyplot as plt

In [10]:
pd.set_option('display.max_columns', None)
# pd.set_option('max_rows', None)

In [5]:
CLEAN_PATH = '../data/data_clean/us_school_districts.csv'

df = pd.read_csv(CLEAN_PATH)

In [6]:
df_2016 = df[(df.YRDATA == 2016)]
df_2016.head()

,STATE,ENROLL,NAME,YRDATA,TOTALREV,TFEDREV,TSTREV,TLOCREV,LOCRTAX,LOCRPROP,LOCREVPAR,PCTTOTAL,PCTFTOT,PCTSTOT,PCTLTOT,TOTALEXP,TCURSPND,TSALWAGE,TEMPBENE,TCURINST,TCURSSVC,PPCSTOT,PPITOTAL,PPSTOTAL
3120,Alabama,9609.0,AUTAUGA COUNTY SCHOOL DISTRICT,2016,80867,7447,53842,19578,6797.0,6775.0,NaN,100.0,9.2,66.6,24.2,76672,74185.0,43030.0,18122.0,43843,23941,7622.0,4563.0,2492.0
3121,Alabama,30931.0,BALDWIN COUNTY SCHOOL DISTRICT,2016,338236,23710,145180,169346,49106.0,48902.0,NaN,100.0,7.0,42.9,50.1,299880,281647.0,156502.0,61621.0,164977,97231,8942.0,5332.0,3143.0
3122,Alabama,912.0,BARBOUR COUNTY SCHOOL DISTRICT,2016,10116,2342,5434,2340,1209.0,1202.0,NaN,100.0,23.2,53.7,23.1,10070,9778.0,5079.0,2064.0,4907,3896,10621.0,5380.0,4272.0
3123,Alabama,2842.0,EUFAULA CITY SCHOOL DISTRICT,2016,26182,3558,15900,6724,2844.0,2836.0,NaN,100.0,13.6,60.7,25.7,29843,25477.0,14015.0,5746.0,15302,7901,8851.0,5384.0,2780.0
3124,Alabama,3322.0,BIBB COUNTY SCHOOL DISTRICT,2016,32486,3664,21846,6976,1716.0,1572.0,NaN,100.0,11.3,67.2,21.5,31662,30616.0,17107.0,7057.0,16407,11087,8931.0,4882.0,3337.0


In [33]:
def n_largest_or_smallest(df, n, operation):
    """Return a sorted df of the n most/least funded districts

        Parameters:
            df (dataframe): the dataframe to be modified
            n (int): the number of districts to keep
            operation (str)['smallest' or 'largest']: whether to return most/least funded 
        Returns:
            new_df (dataframe): A sorted dataframe of the n most/least funded districts in each state
    """
    
    if operation == 'largest':
        new_df = df.groupby(by='STATE')[SCHEMA].apply(lambda x: x.nlargest(n, columns='TOTALREV')) 
    elif operation == 'smallest':
        new_df = df.groupby(by='STATE')[SCHEMA].apply(lambda x: x.nsmallest(n, columns='TOTALREV'))
    else:
        print("operation must be 'largest' or 'smallest'")
        return
    
    return new_df

In [34]:
SCHEMA = ['YRDATA',
'NAME',
'ENROLL',
'TOTALREV',
'TFEDREV',
'TSTREV',
'TLOCREV',
'LOCRTAX',
'LOCRPROP',
#'LOCREVPAR',
'PCTTOTAL',
'PCTFTOT',
'PCTSTOT',
'PCTLTOT',
'TOTALEXP',
'TCURSPND',
'TSALWAGE',
'TEMPBENE',
'TCURINST',
'TCURSSVC',
'PPCSTOT',
'PPITOTAL',
'PPSTOTAL']

least_funded = n_largest_or_smallest(df_2016, 3, 'smallest')
most_funded = n_largest_or_smallest(df_2016, 3, 'largest')

least_funded.head()
most_funded.head()

YRDATA                                       NAME   ENROLL  \
STATE                                                                      
Alabama 3220    2016              MOBILE COUNTY SCHOOL DISTRICT  57581.0   
        3187    2016        JEFFERSON COUNTY BOARD OF EDUCATION  36092.0   
        3121    2016             BALDWIN COUNTY SCHOOL DISTRICT  30931.0   
Alaska  4567    2016                  ANCHORAGE SCHOOL DISTRICT  48324.0   
        4558    2016  MATANUSKA SUSITNA BOROUGH SCHOOL DISTRICT  18742.0   

              TOTALREV  TFEDREV  TSTREV  TLOCREV   LOCRTAX  LOCRPROP  \
STATE                                                                  
Alabama 3220    564939    78217  310545   176177  120352.0   94632.0   
        3187    344274    33131  199350   111793   86396.0   86355.0   
        3121    338236    23710  145180   169346   49106.0   48902.0   
Alaska  4567    758779    78331  463256   217192       0.0       0.0   
        4558    303215    18467  222607    62141       0.0       0.0   

              PCTTOTAL  PCTFTOT  PCTSTOT  PCTLTOT  TOTALEXP  TCURSPND  \
STATE                                                                   
Alabama 3220     100.0     13.8     55.0     31.2    606882  525674.0   
        3187     100.0      9.6     57.9     32.5    338349  321988.0   
        3121     100.0      7.0     42.9     50.1    299880  281647.0   
Alaska  4567     100.0     10.3     61.1     28.6    800058  696207.0   
        4558     100.0      6.1     73.4     20.5    358757  276704.0   

              TSALWAGE  TEMPBENE  TCURINST  TCURSSVC  PPCSTOT  PPITOTAL  \
STATE                                                                     
Alabama 3220  284363.0  119121.0    276941    203328   9037.0    4796.0   
        3187  184880.0   74169.0    179721    112276   8795.0    4980.0   
        3121  156502.0   61621.0    164977     97231   8942.0    5332.0   
Alaska  4567  367144.0  231279.0    411183    263280  14397.0    8509.0   
        4558  138729.0   78139.0    150590    119298  14763.0    8035.0   

              PPSTOTAL  
STATE                   
Alabama 3220    3531.0  
        3187    3111.0  
        3121    3143.0  
Alaska  4567    5448.0  
        4558    6365.0

In [1]:
# df_2016[df_2016.NAME.str.lower().str.contains('ozark', regex=False)]

In [49]:
map_data = gpd.read_file('../data/data_clean/unsd_map/unsd_compiled.shp')

In [50]:
map_data.head()

,STATEFP,UNSDLEA,GEOID,NAME,LSAD,LOGRADE,HIGRADE,MTFCC,SDTYP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,01,00001,0100001,Fort Rucker School District,00,KG,12,G5420,B,E,233059995,2735224,+31.4097368,-085.7458071,"POLYGON ((-85.86572 31.45470, -85.86543 31.454..."
1,01,00003,0100003,Maxwell AFB School District,00,KG,12,G5420,B,E,8476761,566857,+32.3809438,-086.3637490,"POLYGON ((-86.37655 32.39358, -86.37621 32.393..."
2,01,00005,0100005,Albertville City School District,00,KG,12,G5420,None,E,68731806,258708,+34.2631303,-086.2106600,"MULTIPOLYGON (((-86.25104 34.25266, -86.25060 ..."
3,01,00006,0100006,Marshall County School District,00,PK,12,G5420,None,E,1267142967,103647100,+34.3739971,-086.3442702,"MULTIPOLYGON (((-86.10870 34.19063, -86.10873 ..."
4,01,00007,0100007,Hoover City School District,00,KG,12,G5420,None,E,121693900,2594832,+33.3769487,-086.8055789,"MULTIPOLYGON (((-86.66393 33.40902, -86.66333 ..."


In [119]:
sd = df_2016.NAME.str.lower()

sd = sd.str.replace(' school district', '')
sd = sd.str.replace(' sch dist', '')
sd = sd.str.replace(' school dist', '')
sd = sd.str.replace(' schools', '')

sd = sd.str.replace(' co ', '')

# sd = sd.str.replace(' ind ', ' independent ')
# sd = sd.str.replace(' uni ', ' unified ')
# sd = sd.str.replace(' unif', ' unified')

sd = sd.str.replace(" \d+", '', regex=True)
sd = sd.sort_values()
sd

86683                 a e r o special education cooperative
302779                            a w beattie career center
104798                                              a-h-s-t
162796                                             a.c.g.c.
306587                                     abbeville county
372581                                           abbotsford
339966                                           abbott ind
42884                                              abc unif
360572                                             aberdeen
166655                                             aberdeen
62239                                              aberdeen
310574                                           aberdeen-1
339887                                        abernathy ind
340407                                          abilene ind
112078                                      abilene unified
86926                                  abingdon - avon cusd
303155                                  

In [123]:
map_sd = map_data.NAME.str.lower()
map_sd = map_sd.str.replace('school district', '')
map_sd = map_sd.str.replace('sch dist', '')
map_sd = map_sd.str.replace('school dist', '')
map_sd = map_sd.str.replace('schools', '')
map_sd = map_sd.sort_values()
map_sd

6491                                           not defined
7375                                           not defined
1924                                           not defined
1159                                           not defined
7981                                           not defined
8871                                           not defined
3552                                           not defined
4070                                           not defined
4396                                           not defined
5733                                           not defined
8482                                           not defined
288                                            not defined
8513                                           not defined
5292                                           not defined
2214                                           not defined
5391                                           not defined
3007                                           not defin

In [122]:
count = 0

for district in sd:
    tmp = map_sd.str.contains(district, regex=False).any()
    if tmp == 0:
        print(district)
    else:
        count += 1

print(count)
print(count / len(sd))

a e r o special education cooperative
a w beattie career center
aberdeen-1
abingdon - avon cusd
abington hts
abington town schs
absarokee elem-c
absarokee h s dist
acalanes union high
accomack county school
achievement
ackerman elem
acton school department
acushnet town schs
adair county r-2
adams central rural high
adams cheshire reg
adams county special education association
adams five star district
adams twp
adams-arapahoej
adamsboces
addison central supv union
addison northeast supv union
addison northwest supv union
addison town
addison-rutland supv union
adel-desoto-midburn community
adelanto elem
adlai e stevenson district
admiral peary avts
agar-blunt-onida-3
agawam city
agua fria union high
airline community (908)
aitkin ind
ajo and
akin community consolidated
alamance county-burlington
alameda unified
alamedaoffice of ed
alamo town
alamosa rej
alaska gateway regional educational attendance area (reaa)
alba golden ind
albany town
albany town
albany1
albers
albert lea ind
alber

bernards twp
berne knox-westerlo central
berrien regional education service agency
berryessa union elem
bertha hewitt
bertiesch
bertrand village
berwyn
berwyn north
bethalto community unit
bethany community
bethel tate local
bethel town
bethlehem avts
bethlehem center
bethlehem town
bethlehem twp
bethpage uf
bethune r
bettendorf commmunity
bi-county special education cooperative
bicentennial union high
biddeford city
biddle elem dist
big country spec ed coop
big creek elem
big foot high
big fork district
big hollow
big horn1
big horn2
big horn4
big jackson
big lagoon elem
big lake ind
big piney
big sandy elem
big sandy high
big sandyj
big sky school k-12
big sky spec ed coop
big springs elem
big stone city-1
big sur unified
big timber elem dist
bigfork
bighorn3
billerica town sch
billings elem
billings h s dist
birchwood joint
bird island-oliva isd
birney elem dist
bishop cons ind
bismarck community unit
bison-1
bitterroot valley spec ed coop
bitterwater tully union
black butte union
b

center j t
center line city
center moriches uf
center stanton public
centerville r-1
centerville-1
centinela valley union high
central arizona valley institute of technology
central berkshire regional
central california migrant head start jpa
central california school finance authority jpa
central clinton community
central curry #1
central elem spec dist
central hghts ind
central illinois rural region career & technical education sys
central illinois vocational education cooperative
central indiana education service center
central intermediate school unit
central islip uf
central lincoln county school system
central minnesota ecsu
central minnesota joint powers district
central minnesota telecomm district
central montgomery avts
central mt learning resource ctr
central nine vocational technical school
central r-3
central reg high
central savannah river area resa
central sierra rop jpa
central stickney
central susquehanna intermediate school unit
central union elementary
central union h

danbury public
danby town
daniel morgan area vocational center
daniels/sheridan spec ed coop
dansville agr
danvers town sch
danville town
daresch
darien town
darke county educational service center
darrington consolidated
dartmouth public
dassel-cokato publicrict
dauphin county avts
davenport commmunity
davey elem dist
daviesch
daviess-martin special education cooperative
davis jt unif
davis r
dawson -boyd
dawson bryant local
dawson education service coop
dawson high school
dawson indrict
dawson spr ind
dayvillej
de beque jt
de kalb co
de kalb county
de smet-2
de soto c
de soto county
de soto ind
dearborn pub
deblois town
dedham town
dedham town sch
deep river town
deer creek elem dist
deer creek mackinaw community unit
deer isle-stonington community (913)
deer lodge elem dist
deer park city
deer park community consolidated
deer park elem dist
deer park uf
deer trail j
deer- mount judea
deerfield elem dist
deerfield elementary
deerfield town
deerfield town sch dept
deerfield twp
dehesa

etowah county board of education
etowah town
eugenej
eula indrict
eunice public
eureka elem dist
eureka union elem
eureka-1
eustis public
evans brant central
evanston community consolidated
evanston dists/202 jnt agr
evanston township high
evant indrict
eveleth-gilbert isd
everett city sch
evergreen elem
evergreen elem dist
evergreen park
evergreen park community high
evergreen union elem
evesham twp
ewing northern community consolidated
ewing twp
exc children have opportunities
exeter region cooperative
exeter twp
exeter-west greenwich regional
exira community
expeditionary boces
fabens indrict
fair grove r-10
fair haven boro
fair haven town
fair haven union high
fair play r2
fair-mont-egan elem dist
fairbanks n star borough
fairbury city
fairfax elem
fairfax town
fairfaxschs
fairfield com high
fairfield educational service center
fairfield elem
fairfield h s dist
fairfield indrict
fairfield public
fairfield town
fairfield town
fairfield twp
fairfield twp
fairfield-suisun jt unified
f

...

regional education service agency
regional high
regional high
regional management information center
regional management information center
regional management information center
regional management information center
regional management information center
regional management information center
regional office of car/technical education
regional vocational system
reichle elem dist
remer ind
rend lake area regional delivery system
renick r
renssalaer city
rensselaer ctl school corporation
republic county unified
republic michigamme schl dist
republican/twin valley public
rescue elementary
reservation elem
revere city sch
rhinelander city
rhodes-5
rib lake joint
rice cons indrict
rich township high
richards r
richardson indrict
richey elemj
richey h s dist
richfield elem
richfield joint
richfield sprgs ctl
richford town
richgrove elem
richland bean blossom community school corporation
richland county
richland county
richland r-1
richland sprgs ind sch dst
richland-lerdo elementary
richla

sebeka ind
sebewaing-unionville comm
secaucas town
sedgwick town
sedro woolley
seekonk town sch
seeley lake elem dist
seeley union elem
seiad elem
selby-5
selfridge #8
selmaville community consolidated
seminoleinterlocal cooperative
semitropic elem
senath -hornersville c
seneca community consolidated
seneca falls ctl
seneca highlands avts
seneca highlands intermediate school unit
seneca joint
seneca r-7
seneca township high
sequoia union elem
sequoia union high
sevastopol joint
sewanhaka ctl high
seward city
seymour independentrict
seymour joint
seymour r-2
seymour town
shade central city
shady grove
shadyside city
shaffer union elementary
shaftsbury town
shakamak metropolitan
shaker hgts city
shakopee ind
shamong twp
shandon unif
sharon joint
sharon town
sharon town
sharon town sch
shasta union elem
shasta union high
shasta-trinity rop jpa
shastaspl schs oper bysupt
shawmut elem dist
shawnee mission unified
shawnee r
shawsheen valley voc tech
sheepscot valley regional school unit
shel

starpoint ctl
starved rock association for vocational and technical ed
ste genevievesd r2
steamboat springs re2
steel center vocation
steelton highspire
steger
sterling high
sterling town
stevensville elem
stevensville h s dist
steward elementary
stewardson strasburg community unita
stewartstown
stewartsville c2
stidham
stillwater twp
stillwater/sweetgrassspecial ed coop
sto rox
stockbridge central
stockbridge val ctl
stockton city unified
stoddard town
stokessch
stone bank
stone corral elemrict
stoneham town schs
stoney creek jt unif
stonington town
stoughton town schs
stow city
stow creek twp
stowe town
strafford town
strafford town
straight
strain-japan r-16
strasburg franklin
strasburgj
stratford boro
stratford town
stratford town
stratham town
strathmore union elem
stratton r
stratton town
streator elementary
streator township high
strong-huttig
sturbridge town schs
sturgeon r-5
sublette1
success r
sudbury public sch
sudbury town
suffield town
sullivan west consolidated
sulphur bl

waitsfield
wakefield town
wakefield town sch
wakefield twp
walden town
wales public sch
walker-akeley
wall twp
wall-5
wallace community consolidated
walled lake cons school
wallingford town
wallingford village
wallis orchard indrict
wallkill valley regional high
wallowa-region education service dist
walnut bend in d
walnut creek elem
walnut grove elementary district
walnut grove r-5
walnut spgs ind
walpole town schs
walsh re
walthallsch system
waltham city sch
waltham community consolidated
walthill vil
walton verona ind sch d
walworth county children with disabilities education board
walworth elementary joint
wanaque boro
wappinger central
warden-161
wardsboro central
ware public
wareham town schs
warner-5
warren county r-3
warren county special services
warren county vocational school
warren educational service center
warren hills regional
warren township high
warren twp
warren-alvarada-oslo
warrenjt voc
warrensch
warrensville hgts city
warwick city schs
warwick spec
wasatch co
wasco

wynantskill unifed
wyndmere public #42
wyola elem dist
wyoming ctl
wythe county schs
xenia city
yaak elem dist
yamhill-carlton #1
yankton-3
yarmouth town
yarnell
yates center unified
yavapai accommodation
yazoo city mun sep
yellow spgs ex vlg
yellowstone boys&girl el
yellowstone-w/carbonspecial educ coop
yoakum indrict
yolo county special operated bysupt
york county school of technology
york county schs
yorkshire-pioneer ctl
yorkville-ray-dover joint
ypsilanti city
yreka un elem
yreka union high
yubaspl schs oper bysupt
yucaipa jt unified
yucca
yukon flats regional educational attendance area (reaa)
yukon-koyukuk regional educational attendance area (reaa)
yuma elementary
yuma union high
yutan village
zaneis
zapata ind
zeeland city sch
zephyr reverted indrict
zion-benton township high
zumbro educ district
zumbrota-mazepparict
zurich elem dist
7705
0.5378708551483421


In [99]:
print(len(sd))

14325
